
# Build a basic ME model

We will try to build an ME model from the NC_000913.2 Genbank file, the iJO1366 M model, and the complex reconstruction from iJL1650-ME

In [1]:
# python imports
import re
import json
from os.path import join

# third party imports
import pandas
import escher
import cobra.test

# ecoli me
import ecolime
from ecolime.flat_files import *
from ecolime.ecoli_k12 import *
from ecolime import ribosome

from minime import *
from minime.util import building
from minime.util.building import add_ribosome, add_RNA_polymerase
from minime.util.building import add_complex_stoichiometry_data, add_transcription_reaction
from minime.util.building import add_complex_modification_data
from minime.util.building import add_metabolic_reactions
from minime.solve.algorithms import binary_search, fva, solve_at_growth_rate
from minime.solve.symbolic import compile_expressions

Building a full ME model. Run prepare_for_minime() to make minimal model


In [2]:
import cobra.test
ijo = cobra.test.create_test_model('ecoli')

# Colton TODOs 
- 1) look into all of the free modifications in the original ME and make sure none of thes reactions are blocked
- 2) Frameshifts (b2891)
- 3) selenocysteine
- 4) None AUG start codons (15%)
- 5) moaD is a modifier protein that never has to be made

In [3]:
from minime.util.mass import compute_RNA_mass

## Begin by loading metabolites and build Metabolic reactions

In [4]:
me = MEmodel('iJO1366-ME')
m_model = ecolime.get_m_model()
# some of the "metabolites" in iJO1366 "M" model are actually complexes. We pass those in
# so they get created as complexes, not metabolites.
complex_list = []
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("CPLX"))
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("EG"))
complex_list.extend(i.id for i in m_model.metabolites if "-MONOMER" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "-CPLX" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "_mod_" in i.id)
# temp fix
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("Isc"))
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("Suf"))
complex_list = set(complex_list)
building.add_m_model_content(me, m_model, complex_metabolite_ids=complex_list)

# if the bounds of this metabolite aren't open, model uses wrong reactions
me.reactions.EX_pqq_e.lower_bound = -1000

Assigned tl to c
Assigned hemed to c
Assigned pqq to c
Assigned dpm to c
Assigned 23bpg to c
Assigned tqn to c


Sometimes multiple entities can perform the same role. To prevent a combinatorial explosion of possibilities, we can create  "generic" version, where any of those entities can fill in.

In [5]:
for generic, components in get_generics(generic_ions=False).items():
    GenericData(generic, me, components).create_reactions()

## Build ribosome and RNA Polymerase

### Add translation-related Subreactions

In [6]:
for subreaction in ribosome.translation_subreactions:
    subreaction_data = SubreactionData(subreaction, me)
    subreaction_data.enzyme = ribosome.translation_subreactions[subreaction]['enzyme']

In [7]:
gb_filename = join(ecoli_files_dir,'NC_000913.2.gb')                                                                                    
TU_df = pandas.read_csv(join(ecoli_files_dir,'TUs_from_ecocyc.txt'), delimiter="\t", index_col=0)

building.build_reactions_from_genbank(me, gb_filename, TU_df)

/home/sbrg-cjlloyd/minime/minime/util/building.py:205 UserWarning: No TU found for mRNA b0024
/home/sbrg-cjlloyd/minime/minime/util/building.py:205 UserWarning: No TU found for mRNA b4586
/home/sbrg-cjlloyd/minime/minime/util/building.py:205 UserWarning: No TU found for ncRNA b4690
/home/sbrg-cjlloyd/minime/minime/util/building.py:205 UserWarning: No TU found for mRNA b0533
/home/sbrg-cjlloyd/minime/minime/util/building.py:205 UserWarning: No TU found for mRNA b4588
/home/sbrg-cjlloyd/minime/minime/util/building.py:205 UserWarning: No TU found for mRNA b4589
/home/sbrg-cjlloyd/minime/minime/util/building.py:205 UserWarning: No TU found for mRNA b4590
/home/sbrg-cjlloyd/minime/minime/util/building.py:205 UserWarning: No TU found for mRNA b0799
/home/sbrg-cjlloyd/minime/minime/util/building.py:205 UserWarning: No TU found for mRNA b4705
/home/sbrg-cjlloyd/minime/minime/util/building.py:205 UserWarning: No TU found for ncRNA b4417
/home/sbrg-cjlloyd/minime/minime/util/building.py:205 User

TODO deal with selenocystine
b1474 TODO selenocysteine
b1474 TODO selenocysteine
b2891  Needs frameshift?
b2891 TODO selenocysteine
b2891 TODO selenocysteine
b3894 TODO selenocysteine
b3894 TODO selenocysteine
b4079 TODO selenocysteine
b4079 TODO selenocysteine


/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:288 UserWarning: subreaction for codon ACC not added
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:288 UserWarning: subreaction for codon GGU not added
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:288 UserWarning: subreaction for codon AUC not added
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:288 UserWarning: subreaction for codon AAA not added
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:288 UserWarning: subreaction for codon AAC not added
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:288 UserWarning: subreaction for codon AUG not added
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:288 UserWarning: subreaction for codon CGC not added
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:288 UserWarning: subreaction for codon AGC not added
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:288 UserWarning: subreaction for codon ACA not added
/home/sbrg-cjlloyd/minime/minime/core/MEReacti

In [8]:
for metabolite in me.metabolites:
    if metabolite.id == 'generic_tRNA_glu__L_c' or not metabolite.id.startswith('generic_tRNA'):
        continue
    if metabolite.id.replace('generic_tRNA_', '').split('_')[0] == 'START':
        subreaction_data = SubreactionData('fmet_addition_at_START', me)
        #subreaction_data.enzyme = 'InfB_mono' # Start codon loader enzyme
        subreaction_data.stoichiometry = {'h_c': 1, '10fthf_c': -1, 'thf_c':1,
                                         metabolite.id: -1}
    elif metabolite.id.startswith('generic_tRNA'):
        # TODO contrant type is 'trna efficiency' does this change anything?
        # 1 machine + 1 atp + 1 aa + 1 h2o --> 1 machine-amp + 1 h + 1 ppi
        # 1 machine-amp + 1 free tRNA --> 1 machine + 1 amp + 1 charged tRNA
        codon = metabolite.id.replace('generic_tRNA_', '').split('_')[0]
        full_aa = metabolite.id.replace('generic_tRNA_', '').replace(codon + '_', '')
        abreviated_aa = full_aa.split('_')[0]
        subreaction_data = SubreactionData(abreviated_aa + '_addition_at_' + codon, me)
        #subreaction_data.enzyme = 'generic_Tuf' # Default AA loader enzyme
        subreaction_data.stoichiometry = {full_aa: -1, 'atp_c': -1, 'h2o_c': -1,
                                          'amp_c': 1, 'h_c': 1, 'ppi_c': 1,
                                          metabolite.id: -1}

In [9]:
for data in me.translation_data:
    if len(data.nucleotide_sequence) % 3 != 0:
        print data, 'frameshift'
    else:
        codons = (data.nucleotide_sequence[i: i + 3]
                  for i in range(0, (len(data.nucleotide_sequence)-3), 3))
        if 'TGA' in codons:
            print data
    #if not data.compute_sequence_from_DNA.startswith('M'):
    #    pass
        # print data.id, data.nucleotide_sequence[:3], data.compute_sequence_from_DNA[0]

<TranslationData b1474 at 0x7fe7b6e28b50>
<TranslationData b2891 at 0x7fe7b676ded0>
<TranslationData b3894 at 0x7fe7b6254590>
<TranslationData b4079 at 0x7fe7b61aeed0>


In [10]:
building.add_ribosomes_old(me)

add_RNA_polymerase(me)

Created <Metabolite InfA_mono at 0x7fe7b7945390> in <ComplexFormation formation_ribosome at 0x7fe7b79452d0>
Created <Metabolite RimM_mono at 0x7fe7b7454810> in <ComplexFormation formation_ribosome at 0x7fe7b79452d0>
Created <Metabolite Tig_mono at 0x7fe7b74547d0> in <ComplexFormation formation_ribosome at 0x7fe7b79452d0>
Created <Metabolite InfC_mono at 0x7fe7b74540d0> in <ComplexFormation formation_ribosome at 0x7fe7b79452d0>
Created <Metabolite InfB_mono at 0x7fe7b7454090> in <ComplexFormation formation_ribosome at 0x7fe7b79452d0>
Created <Metabolite RbfA_mono at 0x7fe7b7454050> in <ComplexFormation formation_ribosome at 0x7fe7b79452d0>
Created <Metabolite Era_dim at 0x7fe7b7454210> in <ComplexFormation formation_ribosome at 0x7fe7b79452d0>
Created <Metabolite hRNAP at 0x7fe7b5ae1a10> in <ComplexFormation formation_RNA_Polymerase at 0x7fe7b5ae1990>


Make a dummy reactions. Add a dummy protein in as well.

## Associate the tRNA synthetases

The tRNA charging reactions were automatically added when loading the genome from the genbank file. However, the charging reactions still need to be made aware of the tRNA synthetases which are responsible.

In [11]:
with open(join(ecoli_files_dir, "amino_acid_tRNA_synthetase.json"), "rb") as infile:
    aa_synthetase_dict = json.load(infile)
for data in me.tRNA_data:
    data.synthetase = str(aa_synthetase_dict[data.amino_acid])

### Add in complex Formation with modifications

In [12]:
# ME_complex_dict is a dict of {'complex_id': [{'bnum' : count}]}
rna_components = {"b3123"} # component id should have RNA_ instead of protein_
ME_complex_dict = get_complex_to_bnum_dict(rna_components)
# some entries in the file need to be renamed.
# Colton 7/8/15 made changes directly to flat file
#renames = {"MnmE_": "b3706", "MnmG_": "b3741", "YheM_": "b3344", "YheL_": "b3343", "YheN_": "b3345"}
add_complex_stoichiometry_data(me, ME_complex_dict)

In [13]:
rxn_dict = get_reaction_matrix_dict()
met_list = set()
for met_stoich in rxn_dict.values():
    for met in met_stoich:
        if 'mod_c' not in met:
            met_list.add(met.replace('_c',''))
        else:
            met_list.add(met)
met_list.remove('EG10544-MONOMER_mod_palmitate')
modification_dict = get_protein_modification_dict(met_list,  generic=False)
add_complex_modification_data(me, modification_dict)

In [14]:
# two different reactiond can add a lipoate modification.
# We create a separate ModificationData for each one
lipo = me.modification_data.get_by_id("mod_lipo_c")
alt_lipo = ModificationData("mod_lipo_c_alt", me)
#alt_lipo.stoichiometry = lipo.stoichiometry

lipo.stoichiometry = {"lipoamp_c": -1, "amp_c": 1}
lipo.enzyme = 'EG11796-MONOMER'
lipo.keff = 65.

alt_lipo.stoichiometry = {'EG50003-MONOMER_mod_pan4p_mod_lipo':-1,
                          'EG50003-MONOMER_mod_pan4p':1,
                          'h_c':-1,}
alt_lipo.enzyme = 'EG11591-MONOMER'
alt_lipo.keff = 65.

for cplx_data in lipo.get_complex_data():
    alt_cplx_data = ComplexData(cplx_data.id + "alt", me)
    alt_cplx_data.complex_id = cplx_data.complex_id
    alt_cplx_data.translocation = cplx_data.translocation
    alt_cplx_data.chaperones = cplx_data.chaperones
    alt_cplx_data.modifications = cplx_data.modifications
    alt_cplx_data.modifications[alt_lipo.id] = \
        alt_cplx_data.modifications.pop(lipo.id)

In [15]:
# chaperones
# todo bmocogdp_c mods
# todo FE-S modifications
mod_catalysts = {'CPLX0-1762':'G6712-MONOMER', # FE-S modification
                 'TMAOREDUCTI-CPLX':'EG12195-MONOMER', #bmocogdp
                 'DIMESULFREDUCT-CPLX':'G6849-MONOMER', #bmocogdp
                 'NITRATREDUCTA-CPLX':'NARJ-MONOMER', #bmocogdp
                 'NITRATREDUCTZ-CPLX':'NARW-MONOMER', #bmocogdp
                 'NAP-CPLX':'NAPD-MONOMER', #bmocogdp
                 'NAPAB-CPLX_NAPC-MONOMER':'NAPD-MONOMER'} #bmocogdp

## TODO NEXT complete Iron-Sulfur cluster modification reactions. Apply rRNA/tRNA fraction constraint 

In [16]:
essential_list = [
    #'4fe4s_c',
    'LI_c',
    'cs_c',
    #'2fe2s_c',
    '3fe4s_c',
    'palmitate_c',
    'NiFeCoCN2_c', 'acetyl_c',
    'RNase_m5','RNase_m16','RNase_m23'] # RNAses are gaps in model, fix how cs is added to

for met_id in essential_list:
    r = cobra.Reaction("EX_" + met_id)
    me.add_reaction(r)
    r.reaction = met_id + " <=> "

unknown metabolite 'LI_c' created
unknown metabolite '3fe4s_c' created
unknown metabolite 'palmitate_c' created
unknown metabolite 'NiFeCoCN2_c' created
unknown metabolite 'acetyl_c' created
unknown metabolite 'RNase_m5' created
unknown metabolite 'RNase_m16' created
unknown metabolite 'RNase_m23' created


Build all complex formation reactions

In [17]:
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

In [18]:
building.add_dummy_reactions(me, "ATG" + "TTT" * 12 + "TAT" * 12 + "ACG" * 12 + "GAT" * 12 + "AGT" * 12 + "TGA", update=False)

In [19]:
# these guys can transfer assembled iron sulfur clusters to the various enzymes
fes_transfer = {"erpA": "CPLX0-7617", "iscA": "IscA_tetra", "sufA": "CPLX0-7824"}

suf_cplx = ComplexData("sufBC2DES_pathway_complex", me)
suf_cplx.stoichiometry = {"CPLX0-1341": 1, "CPLX0-246_CPLX0-1342_mod_pydx5p": 1}
suf_cplx.create_complex_formation()
    
isc_cplx = ComplexData("iscUS_cyaY_pathway_complex", me) # could add chaperones into here
isc_cplx.stoichiometry = {"IscU": 1, "IscS_mod_2:pydx5p": 1, "EG11653-MONOMER": 1}
isc_cplx.create_complex_formation()

generic_fes_transfer = GenericData("generic_fes_transfer", me, ['CPLX0-7617', 'CPLX0-7824', 'IscA_tetra'])
generic_fes_transfer.create_reactions()

#building.add_and_update_metabolic_reaction(me, "isc_2fe2s_formation", suf_cplx.id, False, update=True)
#building.add_and_update_metabolic_reaction(me, "suf_2fe2s_formation", isc_cplx.id, False, update=True)
#building.add_and_update_metabolic_reaction(me, "isc_4fe4s_formation", suf_cplx.id, False, update=True)
#building.add_and_update_metabolic_reaction(me, "suf_4fe4s_formation", isc_cplx.id, False, update=True)

In [20]:
me.modification_data.mod_2fe2s_c.enzyme = generic_fes_transfer.id
me.modification_data.mod_2fe2s_c.keff = 65.
me.modification_data.mod_4fe4s_c.enzyme = generic_fes_transfer.id
me.modification_data.mod_4fe4s_c.keff = 65.

Crutch reactions for mets that are blocked. TODO remove

## Associate Complexes with Reactions

In [21]:
# associate reaction id with the old ME complex id (including modifications)
rxnToModCplxDict = get_reaction_to_complex(generic=False)

Fixed _DASH:  D__LACtex
Fixed _DASH:  L__LACD2
Fixed _DASH:  L__LACD3
Fixed _DASH:  D__LACt2pp
Fixed _DASH:  L__LACtex
Fixed _DASH:  L__LACt2rpp
Assigned tl to c
Assigned hemed to c
Assigned pqq to c
Assigned dpm to c
Assigned 23bpg to c
Assigned tqn to c


In [22]:
rxn_info = get_reaction_info_frame()
for reaction_data in me.stoichiometric_data:
    add_metabolic_reactions(me, reaction_data, rxnToModCplxDict, rxn_info, update=True,
                            create_new=True)

Rebuild transcription and translation to use tRNA (now that tRNA synthetase complexes are in the model

In [23]:
generic_dict = {'generic_16Sm4Cm1402': ['RsmH_mono', 'RsmI_mono'],
                'generic_LYSINEaaRS': ['LysI_RS_dim', 'LysII_RS_dim_mod_6:mg2'],
                'generic_Dus': ['DusA_mono', 'DusB_mono', 'DusC_mono'],
                'generic_RF': ['PrfA_mono', 'PrfB_mono'],
                'generic_Tuf': ['TufA_mono', 'TufB_mono'],
                'generic_RNase': ['RNase_T_dim_mod_4:mg2', 'RNase_BN_dim_mod_2:zn2',
                                  'Rnd_mono_mod_5:mg2', 'Rnb_mono_mod_1:mg2',
                                  'Rph_mono_mod_mg2']}

#for generic, components in generic_dict.items():
#    components = [R.replace('mg2', 'generic_divalent').
#                  replace('zn2', 'generic_divalent') for R in components]
#    GenericData(generic, me, components).create_reactions()
    
for generic, components in generic_dict.items():
    GenericData(generic, me, components).create_reactions()

In [24]:
for r in me.reactions:
    if isinstance(r, tRNAChargingReaction):
        r.update()
for r in me.reactions:
    if isinstance(r, TranslationReaction):
        r.update()
    if isinstance(r, TranscriptionReaction):
        r.update()
    if isinstance(r, MetabolicReaction):
        r.update()

b1474 TODO selenocysteine
b1474 TODO selenocysteine
b2891 TODO selenocysteine
b2891 TODO selenocysteine
b3894 TODO selenocysteine
b3894 TODO selenocysteine
b4079 TODO selenocysteine
b4079 TODO selenocysteine


In [25]:
apoACP = me.metabolites.get_by_id('EG50003-MONOMER_mod_pan4p')
for reaction in me.metabolites.get_by_id('EG50003-MONOMER_mod_pan4p').reactions:
    if apoACP in reaction.reactants:
        print reaction
        reaction.complex_dilution_list = {apoACP.id}
        reaction.update()
        print reaction.reaction

MCOATA1_FWD_MALONYL-COA-ACP-TRANSACYL-MONOMER
malcoa_c + 4.27350427350427e-6*mu MALONYL-COA-ACP-TRANSACYL-MONOMER + 4.27350427350427e-6*mu + 1.0 EG50003-MONOMER_mod_pan4p --> EG50003-MONOMER_mod_pan4p_mod_mal + coa_c
AACPS31_FWD_AAS-MONOMER_mod_mg2
4.27350427350427e-6*mu AAS-MONOMER_mod_mg2 + hdca_c + 4.27350427350427e-6*mu + 1.0 EG50003-MONOMER_mod_pan4p + atp_c --> amp_c + EG50003-MONOMER_mod_pan4p_mod_palm + ppi_c
AACPS21_FWD_AAS-MONOMER_mod_mg2
ttdcea_c + 4.27350427350427e-6*mu + 1.0 EG50003-MONOMER_mod_pan4p + 4.27350427350427e-6*mu AAS-MONOMER_mod_mg2 + atp_c --> ppi_c + amp_c + EG50003-MONOMER_mod_pan4p_mod_tde
AACPS11_FWD_AAS-MONOMER_mod_mg2
4.27350427350427e-6*mu AAS-MONOMER_mod_mg2 + 4.27350427350427e-6*mu + 1.0 EG50003-MONOMER_mod_pan4p + ttdca_c + atp_c --> amp_c + ppi_c + EG50003-MONOMER_mod_pan4p_mod_myrs
AACPS61_FWD_AAS-MONOMER_mod_mg2
4.27350427350427e-6*mu + 1.0 EG50003-MONOMER_mod_pan4p + ocdca_c + 4.27350427350427e-6*mu AAS-MONOMER_mod_mg2 + atp_c --> EG50003-MONOMER

In [26]:
for reaction in me.metabolites.get_by_id('EG50003-MONOMER_mod_pan4p').reactions:
    reaction.update()
    print reaction
    print reaction.reaction

FA141ACPHi1_FWD_EG11095-MONOMER_mod_zn2_mod_fe2_mod_cobalt2_mod_mg2_mod_mn2
4.27350427350427e-6*mu EG11095-MONOMER_mod_zn2_mod_fe2_mod_cobalt2_mod_mg2_mod_mn2 + h2o_c + EG50003-MONOMER_mod_pan4p_mod_tde --> ttdcea_c + EG50003-MONOMER_mod_pan4p + h_c
MCOATA1_FWD_MALONYL-COA-ACP-TRANSACYL-MONOMER
malcoa_c + 4.27350427350427e-6*mu MALONYL-COA-ACP-TRANSACYL-MONOMER + 4.27350427350427e-6*mu + 1.0 EG50003-MONOMER_mod_pan4p --> EG50003-MONOMER_mod_pan4p_mod_mal + coa_c
3OAS1601_FWD_3-OXOACYL-ACP-SYNTHII-CPLX
4.27350427350427e-6*mu 3-OXOACYL-ACP-SYNTHII-CPLX + EG50003-MONOMER_mod_pan4p_mod_mal + EG50003-MONOMER_mod_pan4p_mod_myrs + h_c --> EG50003-MONOMER_mod_pan4p_mod_3opalm + EG50003-MONOMER_mod_pan4p + co2_c
3OAS1801_FWD_FABB-CPLX
EG50003-MONOMER_mod_pan4p_mod_palm + 4.27350427350427e-6*mu FABB-CPLX + h_c + EG50003-MONOMER_mod_pan4p_mod_mal --> EG50003-MONOMER_mod_pan4p + co2_c + EG50003-MONOMER_mod_pan4p_mod_3ooctd
G3PAT1801_FWD_GLYCEROL-3-P-ACYLTRANSFER-MONOMER
glyc3p_c + EG50003-MONOMER_

## Add in translocation

## Make RNA splicing machinery

In [27]:
#generic_RNase = GenericData("generic_RNase", me,
#                            {ecoli_k12.replace_divalent(i) for i in generic_RNase_list})
#generic_RNase.create_reactions()

rRNA_machinery = ComplexData("rRNA_containing_excision_machinery", me)
#rRNA_machinery.stoichiometry = {ecoli_k12.replace_divalent(i): 1 for i in ecoli_k12.rRNA_containing}
rRNA_machinery.stoichiometry = {i: 1 for i in ecoli_k12.rRNA_containing}
rRNA_machinery.create_complex_formation()
rRNA_mod = ModificationData("rRNA_containing_excision", me)
rRNA_mod.stoichiometry = {'h2o_c': -1, 'h_c': 1}
rRNA_mod.enzyme = rRNA_machinery.id

monocistronic_machinery = ComplexData("monocistronic_excision_machinery", me)
#monocistronic_machinery.stoichiometry = {ecoli_k12.replace_divalent(i): 1 for i in ecoli_k12.monocistronic}
monocistronic_machinery.stoichiometry = {i: 1 for i in ecoli_k12.monocistronic}
monocistronic_machinery.create_complex_formation()
monocistronic_mod = ModificationData("monocistronic_excision", me)
monocistronic_mod.stoichiometry = {'h2o_c': -1, 'h_c': 1}
monocistronic_mod.enzyme = monocistronic_machinery.id

polycistronic_machinery = ComplexData("polycistronic_excision_machinery", me)
#polycistronic_machinery.stoichiometry = {ecoli_k12.replace_divalent(i): 1 for i in ecoli_k12.polycistronic_wout_rRNA}
polycistronic_machinery.stoichiometry = {i: 1 for i in ecoli_k12.polycistronic_wout_rRNA}
polycistronic_machinery.create_complex_formation()
polycistronic_mod = ModificationData("polycistronic_excision", me)
polycistronic_mod.stoichiometry = {'h2o_c': -1, 'h_c': 1}
polycistronic_mod.enzyme = polycistronic_machinery.id

for t in me.transcription_data:
    n_excised = sum(t.excised_bases.values())
    n_cuts = len(t.RNA_products) * 2
    if n_excised == 0 or n_cuts == 0:
        continue
    RNA_types = list(t.RNA_types)
    n_tRNA = RNA_types.count("tRNA")
    
    if "rRNA" in set(RNA_types):
        t.modifications[rRNA_mod.id] = n_cuts
    elif n_tRNA == 1:
        t.modifications[monocistronic_mod.id] = n_cuts
    elif n_tRNA > 1:
        t.modifications[polycistronic_mod.id] = n_cuts
    else:  # only applies to rnpB
        t.modifications[monocistronic_mod.id] = n_cuts

## Remove unused protein and mRNA

This should make the model solve faster

This gives the total number of genes included

## Attempt to set keffs

In [28]:
divalent_list = divalent_list
monovalent_list = ['_mod_k','_mod_na1']
from pickle import load
with open("test_keffs.pickle", "rb") as infile:
    old_keffs = load(infile)
keffs = {}

for keff, value in old_keffs.items():
    keffs[keff] = value

#        for i in divalent_list: 
#        keff = keff.replace(i, 'generic_divalent')
#    for i in monovalent_list: 
#keff = keff.replace(i, '_mod_generic_monovalent')

for r in me.reactions:
    if isinstance(r, MetabolicReaction) and r.complex_data is None:
        continue
    if isinstance(r, MetabolicReaction) and r.complex_data.id != "CPLX_dummy":
        met_rxn = r
        key = met_rxn.id.replace("-", "_DASH_").replace("__", "_DASH_").replace(":","_COLON_")
        #key = met_rxn.id
        key = "keff_" + key.replace("_FWD_", "_").replace("_REV_", "_")

        matches = [i for i in keffs if key in i]
        # get the direction
        if met_rxn.reverse:
            matches = [i for i in matches if i.endswith("_reverse_priming_keff")]
        else:
            matches = [i for i in matches if i.endswith("_forward_priming_keff")]
        if len(matches) == 1:
            met_rxn.keff = keffs[matches[0]]
        elif len(matches) > 0:
            if len(matches) == len([i for i in matches if key + "_mod_"]):
                met_rxn.keff = keffs[matches[0]]
            else:
                print key, len(matches)
        else:  # len(matches) == 0
            print "no keff found for", key

no keff found for keff_GLUTRR_CPLX0_DASH_3741
no keff found for keff_suf_2fe2s_formation_sufBC2DES_pathway_complex
no keff found for keff_suf_4fe4s_formation_sufBC2DES_pathway_complex
no keff found for keff_isc_2fe2s_formation_iscUS_cyaY_pathway_complex
no keff found for keff_isc_4fe4s_formation_iscUS_cyaY_pathway_complex


In [29]:
for r in me.reactions:
    if hasattr(r, "update"):
        r.update()

b1474 TODO selenocysteine
b1474 TODO selenocysteine
b2891 TODO selenocysteine
b2891 TODO selenocysteine
b3894 TODO selenocysteine
b3894 TODO selenocysteine
b4079 TODO selenocysteine
b4079 TODO selenocysteine


In [30]:
for c in me.complex_data:
    try:
        c.create_complex_formation()
        print c
    except:
        pass

In [31]:
for c_d in me.complex_data:
    c = c_d.complex
    if len(c.reactions) == 1:
        list(c.reactions)[0].delete(remove_orphans=True)
        
for p in me.metabolites.query("protein"):
    delete = True
    for rxn in p._reaction:
        if p in rxn.reactants:
            delete = False       
    if delete:
        while len(p._reaction) > 0:
            list(p._reaction)[0].delete(remove_orphans=True)
            
for m in me.metabolites.query("RNA"):
    delete = True
    for rxn in m._reaction:
        if m in rxn.reactants and not rxn.id.startswith('DM_'):
            delete = False       
    if delete:
        me.reactions.get_by_id('DM_' + m.id).remove_from_model(remove_orphans=True)
        m.remove_from_model(method='subtractive')
        
for t in me.reactions.query('transcription_TU'):
    delete = True
    for product in t.products:
        if isinstance(product, TranscribedGene):
            delete = False
    if delete:
        t.remove_from_model(remove_orphans=True)

# TODO address these 
## make sure they are actually dead ends

In [32]:
for metabolites in me.metabolites:
    if len(metabolites.reactions) == 1:
        print metabolites.id, metabolites.reactions

cpg180_c frozenset([<MetabolicReaction CFAS180G_FWD_CFA-CPLX_mod_hco3 at 0x7fe7b7f4c650>])
selnp_c frozenset([<MetabolicReaction SELNPS_FWD_EG10943-MONOMER_mod_mg2 at 0x7fe7b7de0890>])
ragund_c frozenset([<MetabolicReaction O16AT_FWD_EG11984-MONOMER at 0x7fe7b7efeb90>])
23doguln_c frozenset([<MetabolicReaction DOGULNR_FWD_CPLX0-2061 at 0x7fe7b7c36210>])
4mhetz_c frozenset([<MetabolicReaction HETZK_FWD_THZ-KIN-MONOMER at 0x7fe7b75f27d0>])
4hthr_c frozenset([<MetabolicReaction 4HTHRS_FWD_THRESYN-MONOMER_mod_pydx5p at 0x7fe7b79e61d0>])
cpe160_c frozenset([<MetabolicReaction CFAS160E_FWD_CFA-CPLX_mod_hco3 at 0x7fe7b7edcf90>])
gdpfuc_c frozenset([<MetabolicReaction GOFUCR_FWD_CPLX0-3881 at 0x7fe7b7ad2350>])
cinnm_c frozenset([<MetabolicReaction CINNDO_FWD_HCAMULTI-CPLX at 0x7fe7b75e5e90>])
preq1_c frozenset([<MetabolicReaction CDGR_FWD_QueF_dim at 0x7fe7b8844750>])
aconm_c frozenset([<MetabolicReaction ACONMT_FWD_G6806-MONOMER at 0x7fe7b7eecc90>])
didp_c frozenset([<MetabolicReaction NTP11_

## TODO
glycogen_demand = (glycogen_g_per_g)*(1./glycogen_mw)*1000. 

In [33]:
len(me.metabolites.query('RNA_b'))

1550

In [34]:
len(me.reactions.query("transcription"))

1335

In [35]:
demands = {'murein5px4p_p': 0.01389, 'kdo2lipid4_e': 0.01945, 'pe160_c': 0.01786,
           'pe160_p': 0.04594, 'pe161_c': 0.02105, 'pe161_c': 0.05415}
for met, stoich in demands.items():
    r = Reaction('DM_' + met)
    r.add_metabolites({me.metabolites.get_by_id(met): -1})
    r.lower_bound = stoich
    me.add_reaction(r)

## Solve

In [36]:
me.reactions.dummy_reaction_FWD_CPLX_dummy.objective_coefficient = 1.

In [37]:
# Turn off reactions that throw off results as in iOL
KO_list = ['DHPTDNR','DHPTDNRN', 'SUCASPtpp','SUCFUMtpp', 'SUCMALtpp', 'SUCTARTtpp', 
           'CAT', 'FHL', 'SPODM', 'SPODMp']
for reaction in KO_list:
    a = me.reactions.query(reaction + '_')
    for rxn in a:
        rxn.upper_bound = 0
        rxn.lower_bound = 0

In [38]:
me.reactions.EX_glc__D_e.lower_bound = -10

In [39]:
expressions = compile_expressions(me)

In [40]:
solve_at_growth_rate(me, .1, compiled_expressions=expressions)

<Solution 0.10 at 0x7fe7b6c2b250>

In [41]:
binary_search(me, min_mu=0, max_mu=1, mu_accuracy=1e-5,
              compiled_expressions=expressions, debug=False, solver = 'gurobi')

mu		status
0.000000	+
1.000000	-
0.500000	+
0.750000	+
0.875000	+
0.937500	+
0.968750	-
0.953125	-
0.945312	-
0.941406	-
0.939453	-
0.938477	-
0.937988	-
0.937744	+
0.937866	+
0.937927	+
0.937958	+
0.937973	-
0.937965	-
0.937958	+
completed in 15.3 seconds and 20 iterations


<Solution 0.94 at 0x7fe7b6c17510>

In [42]:
me.translation_data.b2020.last_codon

'UAA'

In [43]:
me.reactions.translation_b2020.translation_data.last_codon

'UAA'

In [44]:
ribosome.translation_stop_dict.get('UAA')

'generic_RF'

In [45]:
a.append(me.compute_solution_error())

AttributeError: 'dict' object has no attribute 'id'

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd 
plt.style.use('bmh')
%matplotlib inline

In [ ]:
plt.style.available

In [ ]:
a = pd.DataFrame.from_dict({'uptake': uptake, 'solution': sol_list, 'error':str(error_list)})

In [ ]:
a = a.set_index('uptake')

In [ ]:
sum_error_list = []
lower_bound_error = []
for uptake in error_list:
    sum_error_list.append(uptake['sum_error'])
    lower_bound_error.append(uptake['lower_bound_error'])

In [ ]:
import statistics as stats

In [ ]:
print stats.mean(sum_error_list)
print stats.stdev(sum_error_list)
print stats.mean(lower_bound_error)
print stats.stdev(lower_bound_error)
print max(lower_bound_error)

In [ ]:
error_list

In [ ]:
b = a.plot()
b.set_xlim([0, 25])
b.set_ylabel('$Growth \ rate, \ \mu \ (h^{-1})$')
b.set_xlabel('$Glucose \ uptake \ rate \ bound \ (mmol \ gDW^{-1} h^{-1}$)')
b.legend?
c = b.figure
c.savefig('/home/sbrg-cjlloyd/Desktop/soplex.svg')

In [ ]:
sol.x_dict['ATPM_FWD_CPLX_dummy']

In [ ]:
me.reactions.ATPM_FWD_CPLX_dummy

In [ ]:
error_list

In [ ]:
from matplotlib import pyplot as plt
plt.plot(uptake, sol_list)
plt.show()

In [ ]:
transcription = me.get_transcription_flux()

In [ ]:
{k: v for k, v in me.solution.x_dict.items() if v < -1e-8 or v > 1e-8}

In [ ]:
me.reactions.EX_na1_e.lower_bound

In [ ]:
me.reactions.translation_dummy.reaction

In [ ]:
import escher
view = escher.Builder("iJO1366.Central metabolism")
view.reaction_data = me.get_metabolic_flux()
view.display_in_notebook()

In [ ]:
ppp_shunt = ['G6PDH2r', 'PGL', 'EDA', 'EDD']
for reaction in ppp_shunt:
    for rxn in me.reactions.query(reaction + '_'):
        print reaction, rxn
        print rxn.complex_data.stoichiometry

In [ ]:
upper_gly = ['PGI', 'PFK', 'FBA', 'TPI']
for reaction in upper_gly:
    for rxn in me.reactions.query(reaction + '_F'):
        print reaction, rxn
        print rxn.complex_data.stoichiometry

In [ ]:
me.solution.x_dict['formation_EG50003-MONOMER']